In [8]:
'''
Write a PySpark program to count the occurrences of each word in a given text file. The solution must utilize RDD transformations and actions for processing, and then convert the final RDD into a DataFrame for output. Sort DataFrame by count in descending order.

Hint
To convert an RDD to a DataFrame, use the spark.createDataFrame() method. Example:

df = spark.createDataFrame(rdd, schema=["word", "count"])
Input
The input data will be provided as a plain text file located at /datasets/notes.txt. Each line contains a sentence or phrase. Example input text:

hello world
hello PySpark
PySpark is fun
hello world again
Output
Sample Output Schema
word: String
count: Integer
Example Table
word	count
hello	3
world	2
PySpark	2
is	1
fun	1
again	1
Explanation
The program reads the text file into an RDD.
The RDD is processed to split lines into words, count the occurrences of each word, and sort by word.
The final RDD is converted into a DataFrame with columns word and count and sorted by count in descending order.
Use display(df) to show the final DataFrame.
'''

'''
🚨 When to Use RDD

Use RDD only if:

Data is completely unstructured
Need low-level control
Custom partitioning logic
Legacy Spark code

✅ When to Use DataFrame (Almost Always)

Use DataFrame when:
✔ Working with structured/semi-structured data
✔ Need performance
✔ Writing SQL queries
✔ Using Spark 2.x+

95% of Spark jobs should use DataFrames

what kind of fine grain control rdd provides which is missing in spark?

When people say “RDD gives fine-grained control”, they mean control over execution behavior and 
data handling that DataFrames intentionally hide for optimization and simplicity.

🔍 What “Fine-Grained Control” RDD Provides (That DataFrames Don’t)
1️⃣ Custom Partitioning Logic
RDD

You can define exactly how keys are distributed across partitions.

rdd.partitionBy(
    numPartitions=10,
    partitionFunc=lambda key: hash(key) % 10
)


✔ Control data locality
✔ Reduce shuffle for joins
✔ Optimize skewed keys

DataFrame

❌ No custom partition function
✔ Only repartition() or partitionBy() (coarse-grained)

2️⃣ Control Over Data Serialization
RDD

You control:

Serialization format

Object representation

Custom serializers (Kryo tuning)

spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")


RDD lets you tune object-level serialization.

DataFrame

❌ Serialization is managed internally
✔ Columnar + Tungsten (fast but opaque)

3️⃣ Processing Unstructured / Irregular Data
RDD

Can process:

Free-text

Logs

Binary blobs

Irregular nested formats

rdd.flatMap(lambda line: custom_parse(line))

DataFrame

❌ Needs schema
✔ Best for structured / semi-structured data

4️⃣ Custom Stateful Computation
RDD

You can maintain state inside transformations.

rdd.mapPartitions(lambda it: custom_stateful_logic(it))


✔ Full control per partition
✔ Stateful streaming logic (legacy)

DataFrame

❌ Stateless by design
✔ Only declarative aggregations

5️⃣ Control Over Memory Usage
RDD

You can choose:

Storage level

Serialization strategy

rdd.persist(StorageLevel.MEMORY_AND_DISK_SER)


✔ Fine memory tuning

DataFrame

✔ Can cache
❌ Cannot control serialization details

6️⃣ Side Effects & External Systems
RDD

Allows:

Writing to external systems

Network calls

Custom I/O

rdd.foreachPartition(send_to_api)

DataFrame

❌ Discouraged
✔ Must use structured sinks

7️⃣ Deterministic Execution Order (Within Partition)
RDD

Order is preserved inside partitions.

rdd.mapPartitions(process_in_order)

DataFrame

❌ Order is NOT guaranteed unless explicitly sorted

8️⃣ Custom Error Handling & Retry Logic
RDD

You can implement:

Try/catch per record

Fallback logic

rdd.map(lambda x: safe_parse(x))

DataFrame

❌ Errors fail the whole job
✔ Limited via try_cast (SQL)

⚠ Why Spark Hides This in DataFrames

Spark intentionally removes fine-grained control in DataFrames to:

✔ Enable Catalyst optimization
✔ Enable predicate pushdown
✔ Enable vectorized execution
✔ Reduce developer errors
✔ Improve performance

You trade control for speed and safety.

🎯 Interview-Ready Summary

RDDs provide fine-grained control over data partitioning, serialization, stateful computation, memory management, and custom execution logic.
DataFrames abstract these details to enable powerful optimizations through Catalyst and Tungsten.
RDDs are useful for unstructured data or custom processing, while DataFrames are preferred for most analytics workloads.

'''

# Initialize Spark session
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName('Spark Playground').getOrCreate()

rdd = spark.sparkContext.textFile("./sample_dataset.txt")

word_count_rdd = (
  rdd.flatMap(lambda x : x.split(" ")) # split lines into words
      .map(lambda word : word.strip())             # remove extra spaces
      .filter(lambda word : word != "")            # remove empty words
      .map(lambda word : (word, 1)) # (word, 1)
      .reduceByKey(lambda a, b : a + b) # count words
)

'''
✔ flatMap → converts lines into individual words
✔ map → creates (word, 1) pairs
✔ reduceByKey → efficiently aggregates counts
'''

df = spark.createDataFrame(word_count_rdd, schema=["word", "count"])

df_result = df.sort(F.col("count").desc())

# Display the final DataFrame using the display() function.
df_result.show()

+----------+-----+
|      word|count|
+----------+-----+
|     hello|    4|
|   PySpark|    3|
|        is|    2|
|     world|    2|
|     Spark|    2|
|       fun|    1|
|     makes|    1|
|     again|    1|
|      data|    1|
|       big|    1|
|processing|    1|
|      easy|    1|
|  powerful|    1|
+----------+-----+

